# Pytorch Tutorial

Pytorch is a popular deep learning framework and it's easy to get started.

In [1]:
import torch
import torch.nn as nn
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm
import time
import torch.nn.functional as F

BATCH_SIZE = 128
NUM_EPOCHS = 10

First, we read the mnist data, preprocess them and encapsulate them into dataloader form.

In [2]:
# preprocessing
normalize = transforms.Normalize(mean=[.5], std=[.5])
transform = transforms.Compose([transforms.ToTensor(), normalize])

# download and load the data
train_dataset = torchvision.datasets.MNIST(root='./mnist/', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./mnist/', train=False, transform=transform, download=False)

# encapsulate them into dataloader form
train_loader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
test_loader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)

Then, we define the model, object function and optimizer that we use to classify.

In [3]:
import argparse


parser = argparse.ArgumentParser(description='PyTorch MNIST Example')
parser.add_argument('--batch-size', type=int, default=128, metavar='N',
                        help='input batch size for training (default: 128)')
parser.add_argument('--test-batch-size', type=int, default=1000, metavar='N',
                        help='input batch size for testing (default: 1000)')
parser.add_argument('--epochs', type=int, default=10, metavar='N',
                        help='number of epochs to train (default: 10)')
parser.add_argument('--lr', type=float, default=0.01, metavar='LR',
                        help='learning rate (default: 0.01)')
parser.add_argument('--momentum', type=float, default=0.5, metavar='M',
                        help='SGD momentum (default: 0.5)')
parser.add_argument('--no-cuda', action='store_true', default=False,
                        help='disables CUDA training')
parser.add_argument('--seed', type=int, default=1, metavar='S',
                        help='random seed (default: 1)')
parser.add_argument('--log-interval', type=int, default=10, metavar='N',
                        help='how many batches to wait before logging training status')
parser.add_argument('--save-model', action='store_true', default=False,
                        help='For Saving the current Model')

args = parser.parse_known_args()[0]

torch.manual_seed(args.seed)
use_cuda = not args.no_cuda and torch.cuda.is_available()

device = torch.device("cuda" if use_cuda else "cpu")
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}


class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, 5)
        self.conv2 = nn.Conv2d(10, 20, 5)
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)


def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                       100. * batch_idx / len(train_loader), loss.item()))


def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

model = SimpleNet().to(device)


# TODO:define loss function and optimiter
optimizer = torch.optim.SGD(model.parameters(),lr=args.lr, momentum =args.momentum)



Next, we can start to train and evaluate!

In [4]:
print(device)

cuda


In [5]:
# train and evaluate
for epoch in range(NUM_EPOCHS):
#    for images, labels in tqdm(train_loader):
    train(args, model, device, train_loader, optimizer, epoch)
    test(args, model, device, test_loader)    
    # TODO:forward + backward + optimize
    # evaluate
    # TODO:calculate the accuracy using traning and testing dataset

Train Epoch: 0 [0/60000 (0%)]	Loss: 2.317762
Train Epoch: 0 [1280/60000 (2%)]	Loss: 2.308547
Train Epoch: 0 [2560/60000 (4%)]	Loss: 2.306977
Train Epoch: 0 [3840/60000 (6%)]	Loss: 2.295028
Train Epoch: 0 [5120/60000 (9%)]	Loss: 2.290278
Train Epoch: 0 [6400/60000 (11%)]	Loss: 2.293133
Train Epoch: 0 [7680/60000 (13%)]	Loss: 2.263449
Train Epoch: 0 [8960/60000 (15%)]	Loss: 2.272233
Train Epoch: 0 [10240/60000 (17%)]	Loss: 2.251573
Train Epoch: 0 [11520/60000 (19%)]	Loss: 2.252433
Train Epoch: 0 [12800/60000 (21%)]	Loss: 2.229337
Train Epoch: 0 [14080/60000 (24%)]	Loss: 2.199916
Train Epoch: 0 [15360/60000 (26%)]	Loss: 2.172241
Train Epoch: 0 [16640/60000 (28%)]	Loss: 2.147848
Train Epoch: 0 [17920/60000 (30%)]	Loss: 2.093774
Train Epoch: 0 [19200/60000 (32%)]	Loss: 2.013711
Train Epoch: 0 [20480/60000 (34%)]	Loss: 1.951367
Train Epoch: 0 [21760/60000 (36%)]	Loss: 1.812628
Train Epoch: 0 [23040/60000 (38%)]	Loss: 1.655105
Train Epoch: 0 [24320/60000 (41%)]	Loss: 1.438684
Train Epoch: 0 [

Train Epoch: 3 [26880/60000 (45%)]	Loss: 0.043052
Train Epoch: 3 [28160/60000 (47%)]	Loss: 0.098525
Train Epoch: 3 [29440/60000 (49%)]	Loss: 0.137313
Train Epoch: 3 [30720/60000 (51%)]	Loss: 0.099195
Train Epoch: 3 [32000/60000 (53%)]	Loss: 0.107611
Train Epoch: 3 [33280/60000 (56%)]	Loss: 0.113603
Train Epoch: 3 [34560/60000 (58%)]	Loss: 0.072629
Train Epoch: 3 [35840/60000 (60%)]	Loss: 0.057852
Train Epoch: 3 [37120/60000 (62%)]	Loss: 0.093924
Train Epoch: 3 [38400/60000 (64%)]	Loss: 0.072336
Train Epoch: 3 [39680/60000 (66%)]	Loss: 0.103440
Train Epoch: 3 [40960/60000 (68%)]	Loss: 0.088432
Train Epoch: 3 [42240/60000 (71%)]	Loss: 0.143305
Train Epoch: 3 [43520/60000 (73%)]	Loss: 0.086748
Train Epoch: 3 [44800/60000 (75%)]	Loss: 0.094941
Train Epoch: 3 [46080/60000 (77%)]	Loss: 0.101149
Train Epoch: 3 [47360/60000 (79%)]	Loss: 0.066822
Train Epoch: 3 [48640/60000 (81%)]	Loss: 0.081365
Train Epoch: 3 [49920/60000 (83%)]	Loss: 0.127555
Train Epoch: 3 [51200/60000 (85%)]	Loss: 0.095696


Train Epoch: 6 [53760/60000 (90%)]	Loss: 0.022335
Train Epoch: 6 [55040/60000 (92%)]	Loss: 0.105877
Train Epoch: 6 [56320/60000 (94%)]	Loss: 0.021020
Train Epoch: 6 [57600/60000 (96%)]	Loss: 0.044705
Train Epoch: 6 [58880/60000 (98%)]	Loss: 0.078982

Test set: Average loss: 0.0627, Accuracy: 9779/10000 (98%)

Train Epoch: 7 [0/60000 (0%)]	Loss: 0.094685
Train Epoch: 7 [1280/60000 (2%)]	Loss: 0.047116
Train Epoch: 7 [2560/60000 (4%)]	Loss: 0.068362
Train Epoch: 7 [3840/60000 (6%)]	Loss: 0.032153
Train Epoch: 7 [5120/60000 (9%)]	Loss: 0.105939
Train Epoch: 7 [6400/60000 (11%)]	Loss: 0.019046
Train Epoch: 7 [7680/60000 (13%)]	Loss: 0.040137
Train Epoch: 7 [8960/60000 (15%)]	Loss: 0.057615
Train Epoch: 7 [10240/60000 (17%)]	Loss: 0.053774
Train Epoch: 7 [11520/60000 (19%)]	Loss: 0.078428
Train Epoch: 7 [12800/60000 (21%)]	Loss: 0.034689
Train Epoch: 7 [14080/60000 (24%)]	Loss: 0.041204
Train Epoch: 7 [15360/60000 (26%)]	Loss: 0.039121
Train Epoch: 7 [16640/60000 (28%)]	Loss: 0.031419
Train

#### Q5:
Please print the training and testing accuracy.

In [6]:
test(args, model, device, train_loader)  #Training accuracy is 59009/60000 (98%)
test(args, model, device, test_loader)   #Testing accuracy is 9841/10000 (98%)


Test set: Average loss: 0.0488, Accuracy: 59009/60000 (98%)


Test set: Average loss: 0.0472, Accuracy: 9841/10000 (98%)

